In [40]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe'

In [32]:
dataset_file = "./Data/LME COPPER PRICE.xlsx"
df = pd.read_excel(dataset_file, sheet_name='Sheet1')
df.dropna(inplace=True)

df['month'] = df['Date'].dt.month.astype(int)
df['day_of_month'] = df['Date'].dt.day.astype(int)

# day_of_week=0 corresponds to Monday
df['day_of_week'] = df['Date'].dt.dayofweek.astype(int)
# df['hour_of_day'] = df['Date'].dt.hour.astype(int)

selected_columns = ['Date', 'day_of_week', 'High', 'Low', 'Open', 'Close']
df = df[selected_columns]

df.head()

,Date,day_of_week,High,Low,Open,Close
0,2004-01-02,4,2349.0,2307.0,2307.0,2346.0
1,2004-01-05,0,2385.0,2354.0,2370.0,2376.0
2,2004-01-06,1,2391.0,2320.0,2378.0,2345.0
3,2004-01-07,2,2360.0,2310.0,2335.0,2329.0
4,2004-01-08,3,2425.0,2324.0,2350.0,2424.0


In [3]:
df.set_index('Date', inplace=True)
df['date'] = df.index

datetime_columns = ['date', 'day_of_week', 'High', 'Low', 'Open']
target_column = 'Close'

feature_columns = datetime_columns + ['Close']

# For clarity in visualization and presentation,
# only consider the first 150 hours of data.
df = df[feature_columns]
df.head()

,date,day_of_week,High,Low,Open,Close
Date,,,,,,
2004-01-02,2004-01-02,4,2349.0,2307.0,2307.0,2346.0
2004-01-05,2004-01-05,0,2385.0,2354.0,2370.0,2376.0
2004-01-06,2004-01-06,1,2391.0,2320.0,2378.0,2345.0
2004-01-07,2004-01-07,2,2360.0,2310.0,2335.0,2329.0
2004-01-08,2004-01-08,3,2425.0,2324.0,2350.0,2424.0


In [11]:
plot_length = len(df)
# plot_length = 150
plot_df = df.copy(deep=True).iloc[:plot_length]
plot_df['weekday'] = plot_df['date'].dt.day_name()

fig = px.line(plot_df,
              x="date",
              y="Close",
              # color="weekday",
              title="Copper Price Over Time")
fig.show()

In [13]:
def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i+sequence_length, -1])
    return np.array(X_list), np.array(y_list)

In [14]:
train_split = 0.7
n_train = int(train_split * len(df))
n_test = len(df) - n_train

features = ['day_of_week', 'High', 'Low', 'Open', 'Close']
feature_array = df[features].values

# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=features)

sequence_length = 10
X, y = create_sliding_window(scaled_array,
                             sequence_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

In [20]:
class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):

        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 128 # number of encoder cells (from paper)
        self.hidden_size_2 = 32 # number of decoder cells (from paper)
        self.stacked_layers = 2 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0.5 # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features,
                             self.hidden_size_1,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)

        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        # Important when lstm num_layers > 1
        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred

    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state

    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state

    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

In [25]:
n_features = scaled_array.shape[-1]
sequence_length = 10
output_length = 1

batch_size = 128
n_epochs = 150
learning_rate = 1e-3

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length,
                             batch_size = batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)

In [26]:
bayesian_lstm.train()

for e in range(1, n_epochs+1):
    for b in range(0, len(X_train), batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size]

        X_batch = torch.tensor(features,dtype=torch.float32)
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch)
        loss = criterion(output.view(-1), y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if e % 10 == 0:
      print('epoch', e, 'loss: ', loss.item())

epoch 10 loss:  0.0055957273580133915
epoch 20 loss:  0.0028691748157143593
epoch 30 loss:  0.002242554444819689
epoch 40 loss:  0.0017713351408019662
epoch 50 loss:  0.0010617817752063274
epoch 60 loss:  0.001049292040988803
epoch 70 loss:  0.0008963532745838165
epoch 80 loss:  0.00040437618736177683
epoch 90 loss:  0.0009107652003876865
epoch 100 loss:  0.0004611111944541335
epoch 110 loss:  0.0004423115460667759
epoch 120 loss:  0.00040046300273388624
epoch 130 loss:  0.001349142869003117
epoch 140 loss:  0.00037501248880289495
epoch 150 loss:  0.0001699277199804783


In [27]:
offset = sequence_length

def inverse_transform(y):
  return target_scaler.inverse_transform(y.reshape(-1, 1))

training_df = pd.DataFrame()
training_df['date'] = df['date'].iloc[offset:n_train + offset:1]
training_predictions = bayesian_lstm.predict(X_train)
training_df['Close'] = inverse_transform(training_predictions)
training_df['source'] = 'Training Prediction'

training_truth_df = pd.DataFrame()
training_truth_df['date'] = training_df['date']
training_truth_df['Close'] = df['Close'].iloc[offset:n_train + offset:1]
training_truth_df['source'] = 'True Values'

testing_df = pd.DataFrame()
testing_df['date'] = df['date'].iloc[n_train + offset::1]
testing_predictions = bayesian_lstm.predict(X_test)
testing_df['Close'] = inverse_transform(testing_predictions)
testing_df['source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['date'] = testing_df['date']
testing_truth_df['Close'] = df['Close'].iloc[n_train + offset::1]
testing_truth_df['source'] = 'True Values'

evaluation = pd.concat([training_df,
                        testing_df,
                        training_truth_df,
                        testing_truth_df
                        ], axis=0)

In [35]:
fig = px.line(evaluation,
                 x="date",
                 y="Close",
                 color="source",
                 title="Copper Price Over Time")
fig.show()

In [37]:
n_experiments = 100

test_uncertainty_df = pd.DataFrame()
test_uncertainty_df['date'] = testing_df['date']

for i in range(n_experiments):
  experiment_predictions = bayesian_lstm.predict(X_test)
  test_uncertainty_df['Close_{}'.format(i)] = inverse_transform(experiment_predictions)

Close_df = test_uncertainty_df.filter(like='Close', axis=1)
test_uncertainty_df['Close_mean'] = Close_df.mean(axis=1)
test_uncertainty_df['Close_std'] = Close_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['date', 'Close_mean', 'Close_std']]

/var/folders/pz/_wgxb2_55vq_03wjlfqtmmm80000gn/T/ipykernel_18954/1463710952.py:8: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/pz/_wgxb2_55vq_03wjlfqtmmm80000gn/T/ipykernel_18954/1463710952.py:11: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/pz/_wgxb2_55vq_03wjlfqtmmm80000gn/T/ipykernel_18954/1463710952.py:12: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [39]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['Close_mean'] - 3*test_uncertainty_df['Close_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['Close_mean'] + 3*test_uncertainty_df['Close_std']

In [41]:
test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
test_uncertainty_plot_df = test_uncertainty_plot_df
truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
truth_uncertainty_plot_df = truth_uncertainty_plot_df

upper_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )
lower_trace = go.Scatter(
    x=test_uncertainty_plot_df['date'],
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.1)',
    name='99% Lower Confidence Bound'
    )
real_trace = go.Scatter(
    x=truth_uncertainty_plot_df['date'],
    y=truth_uncertainty_plot_df['Close'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

data = [upper_trace, lower_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for Copper Price Test Data',
                   xaxis_title='Year',
                   yaxis_title='Price (USD)')

fig.show()

In [42]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['Close_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['Close']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:",
      bounds_df['contained'].mean())

Proportion of points contained within 99% confidence interval: 0.950625
